In [ ]:
# 🔧 INSTALL EVERYTHING
!pip install -q diffusers transformers accelerate safetensors gradio_client opencv-python groq

# 🧠 IMPORTS
import torch, json, requests, cv2
import numpy as np
from PIL import Image
from io import BytesIO
from gradio_client import Client, handle_file
from diffusers import StableDiffusionInpaintPipeline
from groq import Groq

# 🔑 GROQ API (LLM)
groq = Groq(api_key="API KEY")  # Replace with your key
llm = groq.chat.completions

def parse_instruction(prompt):
    instruction_prompt = f'''You are an instruction parser.

Instruction: "{prompt}"

Output JSON:
{{"action": "remove" or "replace", "object": "<object>", "target": "<new object>" or null}}

Examples:
remove tiger → {{"action":"remove","object":"tiger","target":null}}
replace tiger with lion → {{"action":"replace","object":"tiger","target":"lion"}}

ONLY return JSON.'''

    response = llm.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": instruction_prompt}],
        temperature=0.0,
        max_completion_tokens=256,
        top_p=1,
        stream=False
    )
    return json.loads(response.choices[0].message.content.strip())

# 🎯 Get mask from Gradio Space (evf-sam2)
def get_mask_from_gradio(image_path, object_prompt):
    client = Client("wondervictor/evf-sam2")
    result_path = client.predict(
        image_np=handle_file(image_path),
        prompt=object_prompt,
        semantic_type=False,
        api_name="/inference_image"
    )
    # 💡 Load directly from file path returned
    vis_image = Image.open(result_path).convert("RGB")
    np_img = np.array(vis_image)

    # Extract mask based on highlight color (blue hues)
    hsv = cv2.cvtColor(np_img, cv2.COLOR_RGB2HSV)
    lower = np.array([100, 50, 50])
    upper = np.array([140, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)

    return Image.fromarray(mask)

def inpaint_with_mask(image, mask, prompt="background"):
    import gc, torch
    from diffusers import StableDiffusionXLInpaintPipeline, StableDiffusionXLImg2ImgPipeline

    image = image.resize((1024, 1024))
    mask = mask.resize((1024, 1024)).convert("L")

    # 🧠 Load Inpainting Pipeline
    pipe = StableDiffusionXLInpaintPipeline.from_pretrained(
        "diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to("cuda")

    # 🖌️ Inpaint
    result = pipe(prompt=prompt, image=image, mask_image=mask).images[0]

    # 🧹 Unload Inpainting to free memory
    del pipe
    torch.cuda.empty_cache()
    gc.collect()

    # 🎨 Load Refiner AFTER unloading inpainting
    refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-refiner-1.0",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to("cuda")

    # ✨ Refine image
    refined = refiner(prompt=prompt, image=result).images[0]

    return refined



# 🔁 MAIN FUNCTION
def run_prompt_edit(image_path, user_prompt):
    original_image = Image.open(image_path).convert("RGB")
    instr = parse_instruction(user_prompt)
    action, obj, tgt = instr["action"], instr["object"], instr.get("target")

    print(f"🧠 Parsed: action={action}, object={obj}, target={tgt}")

    mask = get_mask_from_gradio(image_path, obj)
    inpaint_prompt = tgt if action == "replace" else "background"

    return inpaint_with_mask(original_image, mask, inpaint_prompt)

In [ ]:
from huggingface_hub import login
login()  # will prompt for your token

In [ ]:
import os
from datetime import datetime

# 🖼️ Auto-detect latest image file
def get_latest_image(folder="/content/"):
    images = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not images:
        raise FileNotFoundError("No image files found in the specified folder.")
    latest_image = max(images, key=os.path.getmtime)  # Most recently modified
    return latest_image

# 🎤 Ask user for the task
user_prompt = input("Enter your edit instruction (e.g., 'replace tiger with lion'): ")

# 🏃 Run pipeline
latest_image_path = get_latest_image("/content/")
print(f"📸 Using latest image: {latest_image_path}")

result = run_prompt_edit(latest_image_path, user_prompt)

# 💾 Save & show result
output_path = "/content/edited_image.png"
result.save(output_path)
print(f"✅ Edited image saved at: {output_path}")
result.show()